CNN-LSTM Geothermal BHE Analysis

Repository: [GitHub Project](https://github.com/yourusername/CNN-LSTM_geo)

Files Included:
- `Traindata_geothermal_HybridCNNLSTM_rev10_final.py` - Production code with OE401 correction and DST handling
- `CodeExplanation_JupyterNotebook.ipynb` - This explanatory notebook
- `input/` - Raw sensor data (MeterOE401, MeterOE402, MeterOE403)
- `output/` - Model results and performance plots

To run: `python Traindata_geothermal_HybridCNNLSTM_rev10_final.py`

Predicting Geothermal Heat Pump Performance with Deep Learning

This notebook walks through the analysis performed in the master's thesis on using deep learning to predict how well different types of ground heat exchangers work.

What This Research Is About

The University of Stavanger has a ground-source heat pump system with 120 boreholes drilled 300 meters deep into the ground. Each borehole has pipes that circulate fluid to extract heat from the ground. The question is: can we accurately predict the temperature coming back from these boreholes?

Why does this matter? Better predictions mean better system control, more efficient operation, and lower energy costs.

Three Types of Heat Exchangers

The study compares three different pipe configurations, all at the same depth:

- Single U-tube (45mm diameter): 112 production wells serving the campus
- Double U-tube (45mm diameter): 4 research wells with twice the pipe surface area
- MuoviEllipse (63mm diameter): 4 research wells with an elliptical cross-section

The Model

A CNN-LSTM neural network learns from past temperature, flow rate, and power data to predict future outlet temperatures. The model achieved impressive accuracy: average error of just 0.17°C when predicting temperatures 21 days ahead.

What You'll See

- How the data was loaded and corrected
- Quality checks and cleaning steps
- How all three configurations were combined into one training dataset
- Performance results showing the model works well for all three types
- What these results tell us about geothermal system design

1. Environment Setup and Imports

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

2. Loading the Data

Two Data Issues That Needed Fixing

Timestamp Handling

When daylight saving time transitions happen, some timestamps appear twice in the data. The code uses timezone-aware parsing with `ambiguous=False` and `nonexistent="shift_forward"` to handle these cases correctly.

Overlapping Measurements

The OE401 sensor measures all 120 boreholes together. However, this includes 8 research wells that are also measured separately:
- 4 MuoviEllipse wells (OE402 sensor)
- 4 Double U-tube wells (OE403 sensor)

To get accurate per-well values for the 112 production wells, we need to:
1. Subtract the 8 research wells from the OE401 total
2. Divide by 112 (not 120)

This correction ensures we're not double-counting the research wells.

In [ ]:
def load_sensor_data_with_dst_handling(csv_path, bhe_type, normalize_by_wells=None):
    """
    Load sensor data with comprehensive DST handling.
    
    Parameters:
    -----------
    csv_path : str
        Path to CSV file
    bhe_type : str
        Configuration identifier (single_u45mm, double_u45mm, muovi_ellipse_63mm)
    normalize_by_wells : int or None
        If provided, normalize power and flow by this number (deferred for OE401)
    
    Returns:
    --------
    pd.DataFrame with timezone-aware timestamps
    """
    
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Data file not found: {csv_path}")
    
    print(f"\nLoading {bhe_type} data from: {os.path.basename(csv_path)}")
    
    df = pd.read_csv(csv_path, encoding='utf-8', sep=';', decimal=',')
    print(f"  Raw shape: {df.shape}")
    
    # Step 1: Convert Timestamp from object to datetime with DST handling
    print("  Step 1: Converting timestamps with DST awareness...")
    s = df["Timestamp"].astype(str)
    s_clean = (
        s.str.replace("\u00A0", " ", regex=False)  # remove non-breaking spaces
         .str.strip()
    )
    
    ts_parsed = pd.to_datetime(
        s_clean,
        dayfirst=True,      # Norwegian format: dd.mm.yyyy HH:MM
        errors="coerce"
    )
    
    df["Timestamp"] = ts_parsed
    
    # Step 2: Apply timezone localization with DST transition handling
    print("  Step 2: Localizing to Europe/Oslo timezone with DST resolution...")
    df["Timestamp"] = df["Timestamp"].dt.tz_localize(
        "Europe/Oslo",
        ambiguous=False,            # Fall DST: choose second occurrence (standard time)
        nonexistent="shift_forward" # Spring DST: shift forward to next valid time
    )
    
    # Step 3: Sort by timestamp to ensure temporal order
    print("  Step 3: Sorting by timestamp...")
    df = df.sort_values("Timestamp").reset_index(drop=True)
    
    # Remove duplicate timestamps that might still exist
    initial_len = len(df)
    df = df.drop_duplicates(subset=['Timestamp'], keep='first').reset_index(drop=True)
    if len(df) < initial_len:
        print(f"  Removed {initial_len - len(df)} duplicate timestamps")
    
    # Convert numeric columns
    numeric_cols = ['supply_temp', 'return_temp', 'power_kw', 'flow_rate']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Normalize if specified (for OE402 and OE403, not OE401 yet)
    if normalize_by_wells is not None:
        print(f"  Step 4: Normalizing measurements for {normalize_by_wells} wells...")
        if 'power_kw' in df.columns:
            df['power_kw'] = df['power_kw'] / normalize_by_wells
        if 'flow_rate' in df.columns:
            df['flow_rate'] = df['flow_rate'] / normalize_by_wells
    
    # Add configuration identifier
    df['bhe_type'] = bhe_type
    
    print(f"  Final shape: {df.shape}")
    print(f"  Timestamp range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
    print(f"  Timestamp dtype: {df['Timestamp'].dtype}")
    
    return df

Load Research Well Data First (OE402 and OE403)

We load the research wells first because we need their values to correct the OE401 contamination.

In [ ]:
# Define paths
INPUT_DIR = os.path.join(os.getcwd(), "input")

# Load MuoviEllipse 63mm research wells (4 wells) - OE402
muovi_ellipse_df = load_sensor_data_with_dst_handling(
    csv_path=os.path.join(INPUT_DIR, "MeterOE402_Ellipse63.csv"),
    bhe_type='muovi_ellipse_63mm',
    normalize_by_wells=4
)

# Load Double U45mm research wells (4 wells) - OE403
double_u45_df = load_sensor_data_with_dst_handling(
    csv_path=os.path.join(INPUT_DIR, "MeterOE403_doubleU45.csv"),
    bhe_type='double_u45mm',
    normalize_by_wells=4
)

Load and Correct OE401 Data (Single U45mm Complete Field)

Correction Logic (implemented in load_complete_field_data() in production code):

1. OE401 raw = sum of 120 wells (including 8 research wells)
2. Research wells contribution = OE402 (4 wells) + OE403 (4 wells) = 8 wells
3. Clean production field = OE401 - (OE402 + OE403) aggregated values
4. Per-well value = Clean production field / 112 wells

This removes the contamination and provides accurate per-well measurements for the Single U45mm configuration.

The production script performs this correction by:
- Loading all three sensor files
- Aligning timestamps with DST handling
- Calculating total research contribution: (OE402_perwell × 4) + (OE403_perwell × 4)
- Subtracting research totals from OE401
- Normalizing by 112 production wells

In [ ]:
# Demonstration version for notebook - production code uses load_complete_field_data()
# which implements the full correction within the script

# Load OE401 without normalization yet
single_u45_raw = load_sensor_data_with_dst_handling(
    csv_path=os.path.join(INPUT_DIR, "MeterOE401_singleU45.csv"),
    bhe_type='single_u45mm',
    normalize_by_wells=None  # Don't normalize yet
)

print("\n" + "="*80)
print("CORRECTING OE401 DATA FOR RESEARCH WELL CONTAMINATION")
print("="*80)
print("This demonstrates the correction implemented in load_complete_field_data()")
print("="*80)

# Find overlapping timestamps across all three sensors
common_timestamps = set(single_u45_raw['Timestamp']).intersection(
    set(muovi_ellipse_df['Timestamp'])
).intersection(
    set(double_u45_df['Timestamp'])
)

print(f"\nCommon timestamps across all sensors: {len(common_timestamps):,}")

# Filter to common timestamps
single_u45_aligned = single_u45_raw[single_u45_raw['Timestamp'].isin(common_timestamps)].copy()
muovi_aligned = muovi_ellipse_df[muovi_ellipse_df['Timestamp'].isin(common_timestamps)].copy()
double_aligned = double_u45_df[double_u45_df['Timestamp'].isin(common_timestamps)].copy()

# Sort all by timestamp for alignment
single_u45_aligned = single_u45_aligned.sort_values('Timestamp').reset_index(drop=True)
muovi_aligned = muovi_aligned.sort_values('Timestamp').reset_index(drop=True)
double_aligned = double_aligned.sort_values('Timestamp').reset_index(drop=True)

# Calculate research well aggregated contribution (re-aggregate from per-well back to totals)
# OE402 was divided by 4, OE403 was divided by 4, so multiply back to get totals
research_power_total = (muovi_aligned['power_kw'] * 4) + (double_aligned['power_kw'] * 4)
research_flow_total = (muovi_aligned['flow_rate'] * 4) + (double_aligned['flow_rate'] * 4)

print(f"\nResearch wells total power range: {research_power_total.min():.2f} to {research_power_total.max():.2f} kW")
print(f"Research wells total flow range: {research_flow_total.min():.2f} to {research_flow_total.max():.2f} m³/h")

# Subtract research contribution from OE401 total
single_u45_aligned['power_kw'] = single_u45_aligned['power_kw'] - research_power_total
single_u45_aligned['flow_rate'] = single_u45_aligned['flow_rate'] - research_flow_total

# Now normalize by 112 production wells
single_u45_aligned['power_kw'] = single_u45_aligned['power_kw'] / 112
single_u45_aligned['flow_rate'] = single_u45_aligned['flow_rate'] / 112

print(f"\nCorrected OE401 per-well power range: {single_u45_aligned['power_kw'].min():.3f} to {single_u45_aligned['power_kw'].max():.3f} kW")
print(f"Corrected OE401 per-well flow range: {single_u45_aligned['flow_rate'].min():.3f} to {single_u45_aligned['flow_rate'].max():.3f} m³/h")

# Use corrected data
single_u45_df = single_u45_aligned

print("\n" + "="*80)
print("DATA LOADING COMPLETE WITH CORRECTIONS")
print("="*80)
print(f"\nSingle U45mm (112 production wells): {len(single_u45_df):,} records")
print(f"Double U45mm (4 research wells): {len(double_u45_df):,} records")
print(f"MuoviEllipse 63mm (4 research wells): {len(muovi_ellipse_df):,} records")

3. Checking Data Quality

Before cleaning, let's look at what we're working with for each configuration.

In [ ]:
def assess_data_quality(df, config_name):
    """Generate comprehensive data quality report."""
    
    print(f"\n{'='*60}")
    print(f"Data Quality Assessment: {config_name}")
    print(f"{'='*60}")
    
    print(f"\nShape: {df.shape}")
    print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
    print(f"Duration: {(df['Timestamp'].max() - df['Timestamp'].min()).days} days")
    
    print("\nMissing values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    for col, count in missing.items():
        if count > 0:
            print(f"  {col}: {count} ({missing_pct[col]}%)")
    
    print("\nDescriptive statistics:")
    numeric_cols = ['supply_temp', 'return_temp', 'power_kw', 'flow_rate']
    available_cols = [col for col in numeric_cols if col in df.columns]
    print(df[available_cols].describe().round(3))
    
    return df

# Assess quality for all configurations
assess_data_quality(single_u45_df, "Single U45mm (112 production wells - corrected)")
assess_data_quality(double_u45_df, "Double U45mm (4 research wells)")
assess_data_quality(muovi_ellipse_df, "MuoviEllipse 63mm (4 research wells)")

4. Cleaning the Data

The data goes through 8 cleaning steps to remove sensor errors and measurement glitches:

1. Remove physically impossible values (e.g., temperatures below -10°C or above 50°C)
2. Detect stuck sensors (same value repeated 18+ times)
3. Apply median filtering to smooth out noise
4. Fill small gaps (up to 20 minutes) with interpolation
5. Check that temperature and power measurements make physical sense together
6. Remove records with too many missing values
7. Final cleanup

This keeps the real thermal behavior while filtering out bad measurements.

In [ ]:
def clean_bhe_data(df, dataset_name=""):
    """
    8-stage data cleaning pipeline from thesis methodology.
    """
    
    if len(df) == 0:
        return df
    
    print(f"\n{'='*60}")
    print(f"Cleaning: {dataset_name}")
    print(f"Initial records: {len(df):,}")
    print(f"{'='*60}")
    
    df_clean = df.copy()
    
    temp_cols = ['supply_temp', 'return_temp']
    power_cols = ['power_kw']
    flow_cols = ['flow_rate']
    
    # Stage 2: Physical constraint validation
    print("\nStage 2: Physical constraint validation")
    for col in temp_cols:
        if col in df_clean.columns:
            invalid = ((df_clean[col] < -10) | (df_clean[col] > 50))
            if invalid.sum() > 0:
                print(f"  {col}: Removed {invalid.sum()} out-of-range values")
                df_clean.loc[invalid, col] = np.nan
    
    for col in power_cols:
        if col in df_clean.columns:
            invalid = ((df_clean[col] < -500) | (df_clean[col] > 500))
            if invalid.sum() > 0:
                print(f"  {col}: Removed {invalid.sum()} out-of-range values")
                df_clean.loc[invalid, col] = np.nan
    
    for col in flow_cols:
        if col in df_clean.columns:
            invalid = ((df_clean[col] < 0) | (df_clean[col] > 100))
            if invalid.sum() > 0:
                print(f"  {col}: Removed {invalid.sum()} out-of-range values")
                df_clean.loc[invalid, col] = np.nan
    
    # Stage 3: Sensor anomaly detection (stuck readings)
    print("\nStage 3: Sensor anomaly detection (18+ consecutive duplicates)")
    for col in temp_cols + power_cols + flow_cols:
        if col in df_clean.columns:
            mask = df_clean[col].rolling(window=18, center=False).apply(
                lambda x: len(set(x)) == 1, raw=True
            ).fillna(0) > 0
            if mask.sum() > 0:
                print(f"  {col}: Flagged {mask.sum()} stuck sensor readings")
                df_clean.loc[mask, col] = np.nan
    
    # Stage 4: Median filtering
    print("\nStage 4: Median filtering")
    window_temp = 3
    window_other = 5
    
    for col in temp_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].rolling(window=window_temp, center=True, min_periods=1).median()
    
    for col in power_cols + flow_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].rolling(window=window_other, center=True, min_periods=1).median()
    
    print(f"  Applied median filtering (temps={window_temp}, other={window_other})")
    
    # Stage 5: Gap interpolation (max 4 consecutive readings = 20 minutes)
    print("\nStage 5: Gap interpolation (max 4 consecutive readings)")
    max_gap = 4
    for col in temp_cols + power_cols + flow_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].interpolate(method='linear', limit=max_gap, limit_direction='forward')
    
    # Stage 6: Physics validation (temperature-power consistency)
    print("\nStage 6: Physics validation (temperature-power consistency)")
    if all(col in df_clean.columns for col in ['supply_temp', 'return_temp', 'power_kw']):
        temp_diff = df_clean['supply_temp'] - df_clean['return_temp']
        
        # Heat extraction: power > 0, return should be cooler (temp_diff > 0, but not excessive)
        invalid_extraction = (df_clean['power_kw'] > 0) & ((temp_diff < 0) | (temp_diff > 8))
        
        # Heat rejection: power < 0, return should be warmer (temp_diff < 0, but not excessive)
        invalid_rejection = (df_clean['power_kw'] < 0) & ((temp_diff > 0) | (temp_diff < -8))
        
        invalid_physics = invalid_extraction | invalid_rejection
        
        if invalid_physics.sum() > 0:
            print(f"  Removed {invalid_physics.sum()} physics-inconsistent readings")
            df_clean.loc[invalid_physics, ['supply_temp', 'return_temp', 'power_kw']] = np.nan
    
    # Stage 7: Missing data threshold filtering
    print("\nStage 7: Missing data threshold filtering")
    essential_cols = temp_cols + power_cols + flow_cols
    available_cols = [col for col in essential_cols if col in df_clean.columns]
    
    if dataset_name.lower().startswith('single'):
        missing_threshold = 0.25
    else:
        missing_threshold = 0.50
    
    if available_cols:
        missing_pct = df_clean[available_cols].isnull().sum(axis=1) / len(available_cols)
        rows_to_drop = missing_pct > missing_threshold
        if rows_to_drop.sum() > 0:
            print(f"  Removed {rows_to_drop.sum()} rows exceeding {missing_threshold*100}% missing threshold")
            df_clean = df_clean[~rows_to_drop].copy()
    
    # Stage 8: Final quality assurance
    print("\nStage 8: Final quality assurance")
    initial_len = len(df_clean)
    df_clean = df_clean.dropna(subset=available_cols).copy()
    final_removed = initial_len - len(df_clean)
    
    if final_removed > 0:
        print(f"  Removed {final_removed} rows with remaining NaN values")
    
    retention_pct = len(df_clean) / len(df) * 100
    print(f"\nFinal: {len(df_clean):,} records ({retention_pct:.1f}% retained)")
    
    return df_clean

# Apply cleaning to all configurations
single_u45_clean = clean_bhe_data(single_u45_df, "Single U45mm (112 production wells)")
double_u45_clean = clean_bhe_data(double_u45_df, "Double U45mm (4 research wells)")
muovi_ellipse_clean = clean_bhe_data(muovi_ellipse_df, "MuoviEllipse 63mm (4 research wells)")

5. Temperature Distribution Plots

These plots show supply vs return temperatures for each configuration after cleaning. Clean data should show a clear relationship without extreme outliers.

In [ ]:
# Display temperature distribution plots from output folder
output_dir = 'output'

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Temperature Distributions by Configuration', fontsize=16, fontweight='bold')

image_files = [
    ('singleu45_(oe401)_temperature_distribution.png', 'Single U-tube 45mm'),
    ('doubleu45_(oe403)_temperature_distribution.png', 'Double U-tube 45mm'),
    ('muoviellipse_(oe402)_temperature_distribution.png', 'MuoviEllipse 63mm')
]

for ax, (img_file, title) in zip(axes, image_files):
    img_path = os.path.join(output_dir, img_file)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(title, fontsize=12, fontweight='bold')
    else:
        ax.text(0.5, 0.5, f'Image not found:\n{img_file}', 
                ha='center', va='center', transform=ax.transAxes)
        ax.axis('off')

plt.tight_layout()
plt.show()


print("Each plot shows how supply and return temperatures relate for that configuration.")print("The tight clustering indicates good data quality after cleaning.")

6. Combining All Configurations

Why Train One Model Instead of Three?

All three configurations share the same basic physics: heat flowing through pipes into the ground. By training one model on all the data:

- The model learns these shared patterns from more data (especially the 112 production wells)
- A `bhe_type` number (0, 1, or 2) tells the model which configuration it's looking at
- The model can learn what's different about each configuration while using the shared patterns

This is more efficient than training three separate models, and the results show it works well.

In [ ]:
# Find overlapping time period across all configurations
start_time = max(
    single_u45_clean['Timestamp'].min(),
    double_u45_clean['Timestamp'].min(),
    muovi_ellipse_clean['Timestamp'].min()
)

end_time = min(
    single_u45_clean['Timestamp'].max(),
    double_u45_clean['Timestamp'].max(),
    muovi_ellipse_clean['Timestamp'].max()
)

print(f"Overlapping operational period:")
print(f"  Start: {start_time}")
print(f"  End: {end_time}")
print(f"  Duration: {(end_time - start_time).days} days")

# Filter to overlapping period
single_u45_overlap = single_u45_clean[
    (single_u45_clean['Timestamp'] >= start_time) & 
    (single_u45_clean['Timestamp'] <= end_time)
].copy()

double_u45_overlap = double_u45_clean[
    (double_u45_clean['Timestamp'] >= start_time) & 
    (double_u45_clean['Timestamp'] <= end_time)
].copy()

muovi_ellipse_overlap = muovi_ellipse_clean[
    (muovi_ellipse_clean['Timestamp'] >= start_time) & 
    (muovi_ellipse_clean['Timestamp'] <= end_time)
].copy()

# Combine into unified dataset
combined_df = pd.concat([
    single_u45_overlap,
    double_u45_overlap,
    muovi_ellipse_overlap
], ignore_index=True)

# Sort by timestamp
combined_df = combined_df.sort_values('Timestamp').reset_index(drop=True)

print(f"\nUnified dataset shape: {combined_df.shape}")
print(f"\nConfiguration distribution:")
print(combined_df['bhe_type'].value_counts())
print(f"\nSample distribution:")
for bhe_type in combined_df['bhe_type'].unique():
    count = (combined_df['bhe_type'] == bhe_type).sum()
    pct = count / len(combined_df) * 100
    print(f"  {bhe_type}: {count:,} samples ({pct:.1f}%)")

print("\nThis unified dataset is the foundation for training a single model across all configurations.")

7. Features Used for Prediction

The model uses four pieces of information to predict outlet temperature:

1. Supply temperature: How warm is the fluid going into the ground?
2. Flow rate: How fast is the fluid moving? (per well)
3. Power: How much heat is being extracted\rejected? (per well)
4. Configuration type: Which pipe configuration is this? (0=Single, 1=Double, 2=Ellipse)

The model looks at the past 4 hours (48 five-minute readings) to predict the next temperature value.

Target: Return temperature from the borehole

In [ ]:
# Encode BHE types as categorical numerical values
bhe_encoding = {
    'single_u45mm': 0,
    'double_u45mm': 1,
    'muovi_ellipse_63mm': 2
}

combined_df['bhe_type_encoded'] = combined_df['bhe_type'].map(bhe_encoding)

print("BHE Type Encoding:")
for bhe_type, code in bhe_encoding.items():
    print(f"  {bhe_type}: {code}")

# Define feature columns for model input
feature_cols = ['supply_temp', 'flow_rate', 'power_kw', 'bhe_type_encoded']
target_col = 'return_temp'

print(f"\nInput features: {feature_cols}")
print(f"Target variable: {target_col}")

# Display feature statistics by configuration
print("\nFeature statistics by configuration:")
summary = combined_df.groupby('bhe_type')[['supply_temp', 'return_temp', 'power_kw', 'flow_rate']].describe()
print(summary.round(3))

print("\nNote: Different configurations show distinct operational characteristics,")
print("confirming that the model must learn configuration-specific behaviors.")

8. Model Performance Results

The comprehensive analysis below shows how well the model predicts temperatures for all three configurations over a 21-day test period.

What to look for:
- The predicted line should follow the actual measurements closely
- MAE (Mean Absolute Error) shows average prediction error in °C
- All three configurations maintain errors well below 0.2°C
- Different configurations show slightly different accuracy, which makes sense given their different designs

In [ ]:
# Display the comprehensive collector analysis from thesis output
try:
    img_path = os.path.join('output', 'comprehensive_collector_analysis.png')
    if os.path.exists(img_path):
        img = Image.open(img_path)
        plt.figure(figsize=(18, 14))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Comprehensive Collector Analysis - Unified Model Performance', 
                 fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        print("\nPerformance Metrics from Unified Model (21-day test window):")
        print("  Single U45mm: MAE ≈ 0.174°C, RMSE ≈ 0.238°C")
        print("  Double U45mm: MAE ≈ 0.164°C, RMSE ≈ 0.227°C")
        print("  MuoviEllipse 63mm: MAE ≈ 0.178°C, RMSE ≈ 0.241°C")
        print("  Overall: MAE ≈ 0.172°C, RMSE ≈ 0.235°C")
        print("\nThese differentiated performance metrics demonstrate that the unified model")
        print("successfully learns configuration-specific thermal behaviors while maintaining")
        print("consistent sub-degree accuracy across all BHE types.")
    else:
        print(f"Comprehensive analysis plot not found at: {img_path}")
        print("Run the main training script first to generate this output.")
except Exception as e:
    print(f"Error loading image: {e}")

9. What This Means

Key Takeaways

Data Quality Matters
Correcting the OE401 overlap and handling timestamp issues properly were essential for getting accurate results. Without these corrections, the production well data would have been skewed by including the research wells twice.

One Model, Three Configurations
The model achieves sub-degree accuracy (average error 0.17°C) for all three configurations. This proves that training one unified model works well, even though each configuration behaves somewhat differently.

Performance Differences
- Double U-tube: Best prediction accuracy (0.164°C average error)
- Single U-tube: Solid baseline performance with most data
- MuoviEllipse: Slightly higher error but still excellent (0.178°C)

Why This Approach Works

The CNN-LSTM architecture captures both spatial patterns (through CNN layers) and time-series patterns (through LSTM layers). Adding the configuration type as an input feature lets the model adjust its predictions for each design while sharing the learned thermal physics across all of them.

Practical Benefits
- One model is simpler to deploy and maintain than three
- The large production field dataset helps improve predictions for the smaller research configurations
- Sub-degree accuracy is excellent for geothermal system control and optimization

10. Running the Full Training

The production script `Traindata_geothermal_HybridCNNLSTM_rev10_final.py` contains all the corrections and complete implementation.

To train the model from scratch:

```bash
python Traindata_geothermal_HybridCNNLSTM_rev10_final.py
```

What it does:
- Loads all three datasets with OE401 correction and DST handling
- Applies 8-stage data cleaning pipeline
- Trains the CNN-LSTM model (up to 50 epochs with early stopping)
- Generates comprehensive performance plots
- Saves trained model and metrics

Key Differences from Notebook:

This notebook demonstrates the concepts, but the production script includes:
- Complete 8-stage cleaning pipeline (shown simplified here)
- Full CNN-LSTM architecture with optimized hyperparameters
- GPU memory optimization and batch size tuning
- Comprehensive visualization functions for all metrics
- Proper train/validation/test splitting with time-based windows

Model Architecture:
- Input: 4 hours of data (48 five-minute intervals) × 4 features
- CNN layers: Extract spatial patterns (32, 64 channels)
- LSTM layers: Capture time-series dependencies (64 hidden units, 2 layers)
- Output: Next timestep return temperature

Output Files:
- `output/comprehensive_collector_analysis.png` - Main results figure with forecast windows
- `output/collector_configuration_performance_analysis.png` - Collector performance results
- `output/comprehensive_model.pth` - Trained model weights
- `output/comprehensive_results.json` - Detailed performance metrics
- `output/comprehensive_analysis.log` - Complete training log

Training typically takes 15-30 minutes on a GPU, depending on early stopping.

Repository Structure for GitHub:
```
CNN-LSTM_geo/
├── Traindata_geothermal_HybridCNNLSTM_rev10_final.py  # Master project code
├── CodeExplanation_JupyterNotebook.ipynb              # Explanatory notebook
├── input/                                              # Raw sensor data
│   ├── MeterOE401_singleU45.csv
│   ├── MeterOE402_Ellipse63.csv
│   └── MeterOE403_doubleU45.csv
└── output/                                             # Generated results
    ├── comprehensive_collector_analysis.png
    ├── collector_configuration_performance_analysis.png
    ├── comprehensive_model.pth
    ├── comprehensive_results.json
    └── comprehensive_analysis.log
```